In [11]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import csv
from tqdm.notebook import tqdm

In [9]:
# read the urls from the txt file
file = open('/content/urls.txt','r')
ad_urls_list = file.readlines()

In [7]:
ad_urls_list[1]

'https://www.dubizzle.com.eg/en/ad/%D8%B4%D9%82%D9%87-%D9%84%D9%84%D8%A7%D9%8A%D8%AC%D8%A7%D8%B1-%D8%A8%D9%83%D9%85%D8%A8%D9%88%D9%86%D8%AF-%D9%83%D8%A7%D9%8A%D8%B1%D9%88-%D9%81%D9%8A%D8%B3%D8%AA%D9%81%D8%A7%D9%84-%D8%B3%D9%8A%D8%AA%D9%89-ID198575185.html\n'

In [10]:
apartments_data = []

In [13]:
# check for duplicated URLs
seen = set()
uniq_url = []
for url in ad_urls_list:
    if url not in seen:
        uniq_url.append(url)
        seen.add(url)
print("Original list:", len(ad_urls_list))
print("Unique list:", len(uniq_url))

Original list: 10000
Unique list: 9641


In [14]:
def get_apartments_details(all_apartments_url:list):
    for apartment_url in tqdm(all_apartments_url):
        features = {}
        data = requests.get(apartment_url).content
        soup = BeautifulSoup(data , 'lxml')

        description = soup.find('div', {'class': '_0f86855a'})
        # fix: AttributeError: 'NoneType' object has no attribute 'find'
        if description is None:
            features['Description'] = np.nan
        else:
            description = description.find('span').text.strip()
            features['Description'] = description

        details = soup.find('div', {'class': '_241b3b1e'})
        if details is None:
            continue
        else:
            details = details.find_all('div', {'class': '_676a547f'})
        for d in details:
            features[d.find_all('span')[0].text.strip()] = d.find_all('span')[1].text.strip()

        location = soup.find('div', {'class': '_1075545d d059c029 _858a64cf'}) \
            .find('span', {'aria-label': 'Location'}).text.strip()
        # fix: IndexError: list index out of range
        if len(re.findall('(^.*)\,', location)) == 0: # if area is empty
            area = location
            city = location
        else:
            area = re.findall('(^.*)\,', location)[0]
            city = re.findall(',\s(.*)', location)[0]

        features['Area'] = area
        features['City'] = city

        # =================================================================================
        amenities = soup.find('div', {'class': '_4ab34fd4'})
        total_amenities = ['Electricity Meter', 'Built in Kitchen Appliances', 'Water Meter', 'Elevator', 'Balcony', 'Natural Gas', 'Landline', 'Security', 'Central A/C & heating', 'Covered Parking', 'Pets Allowed', 'Private Garden', 'Maids Room', 'Pool']

        if amenities is not None:
            amenities = amenities.find_all('span', {'class': '_66b85548'})
            amenities_list = []
            for amenity in amenities:
                amenities_list.append(amenity.text.strip())
            for amenity in total_amenities:
                features[amenity] = 'Yes' if amenity in amenities_list else 'No'
        else:
            for amenity in total_amenities:
                features[amenity] = np.nan

        apartments_data.append(features)

In [ ]:
get_apartments_details(uniq_url)

In [17]:
df = pd.DataFrame(apartments_data)

In [18]:
df[df.duplicated()]

,Description,Type,Price,Price Type,Video,Area (m²),Bedrooms,Bathrooms,Level,Furnished,...,Landline,Security,Central A/C & heating,Covered Parking,Pets Allowed,Private Garden,Maids Room,Pool,Compound,Down Payment
1410,لعشاق التميز و الراحة و الهدوء،\nشقه مفروشه لل...,Apartment,"30,000",Price,Not Available,90,2,1,1,Yes,...,Yes,Yes,Yes,No,No,No,No,No,NaN,NaN
1923,شقق فندقيه للايجار بالإسكندرية\nبالمندره بجوار...,Apartment,700,Price,Not Available,110,3,2,10+,Yes,...,No,No,No,No,No,No,No,No,NaN,NaN
2815,للايجار مفروش اول سكن 224 متر برحاب2 \n3نوم 3ح...,Apartment,"67,000",Negotiable,Not Available,224,3,3,1,Yes,...,Yes,Yes,Yes,Yes,Yes,No,No,No,NaN,NaN
2816,للايجار مفروش اول سكن 224 متر برحاب2 \n3نوم 3ح...,Apartment,"67,000",Negotiable,Not Available,224,3,3,1,Yes,...,Yes,Yes,Yes,Yes,Yes,No,No,No,NaN,NaN
3587,للإيجار شقة مميزة جدا فى جاردينيا سيتي مدينة ن...,Apartment,"25,000",Price,Not Available,120,3,1,1,Yes,...,No,No,No,No,No,No,No,No,NaN,NaN
3623,متاح للايجار المفروش \nارضي بحديقه 3غرف نوم وح...,Apartment,"25,000",Price,Not Available,116,3,2,Ground,Yes,...,Yes,Yes,No,Yes,No,Yes,No,No,NaN,"25,000"
4401,Apartment for rent in new Giza carnell\n223m²\...,Apartment,"22,000",Price,Not Available,223,3,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4876,- شقه 100 م للايجار المفروش سموحه تانى نمره من...,Apartment,"10,000",Price,Not Available,100,2,1,9,Yes,...,No,Yes,No,No,No,No,No,No,NaN,NaN
6065,شقه 220 مساحه صافيه فيو مميزبجوار مسجد حسن الش...,Apartment,"15,000",Price,Not Available,220,3,2,2,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7885,شقه للايجار ف فورتى ويست سوديك 154م بالتكييفات...,Apartment,"35,000",Price,Not Available,154,2,3,1,No,...,No,Yes,Yes,No,No,No,No,No,NaN,NaN


In [19]:
df = df.drop_duplicates()

In [20]:
df.shape

(9612, 29)

In [21]:
df.to_csv('/content/sample_data/houses.csv', index=False)